In [1]:

# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("SparkByExamples.com") \
      .getOrCreate() 


In [2]:
spark

# DataFrame from RDD

In [3]:
from datetime import date
a=[
    (1,1.0,'shafi',date(2022,1,19)),
    (3,4.0,'ravi',date(2022,12,31)),
     (4,5.0,'karan',date(2022,11,1)),
     (2,14.78,'nadeem',date(2022,11,4)),
     (22,13.0,'mir',date(2022,7,9))
  ]
cols=[('sno' ,int) ,('gpa', float),('name', str),('doj' ,date)]

In [4]:
a

[(1, 1.0, 'shafi', datetime.date(2022, 1, 19)),
 (3, 4.0, 'ravi', datetime.date(2022, 12, 31)),
 (4, 5.0, 'karan', datetime.date(2022, 11, 1)),
 (2, 14.78, 'nadeem', datetime.date(2022, 11, 4)),
 (22, 13.0, 'mir', datetime.date(2022, 7, 9))]

In [5]:
rdd2=spark.sparkContext.parallelize(a)
type(rdd2)

pyspark.rdd.RDD

In [6]:
rdd2.collect()

[(1, 1.0, 'shafi', datetime.date(2022, 1, 19)),
 (3, 4.0, 'ravi', datetime.date(2022, 12, 31)),
 (4, 5.0, 'karan', datetime.date(2022, 11, 1)),
 (2, 14.78, 'nadeem', datetime.date(2022, 11, 4)),
 (22, 13.0, 'mir', datetime.date(2022, 7, 9))]

In [7]:
df=spark.createDataFrame(rdd2,schema=["num1","float1","str1","date1"])
df

DataFrame[num1: bigint, float1: double, str1: string, date1: date]

In [8]:
df.show()

+----+------+------+----------+
|num1|float1|  str1|     date1|
+----+------+------+----------+
|   1|   1.0| shafi|2022-01-19|
|   3|   4.0|  ravi|2022-12-31|
|   4|   5.0| karan|2022-11-01|
|   2| 14.78|nadeem|2022-11-04|
|  22|  13.0|   mir|2022-07-09|
+----+------+------+----------+



In [9]:
df.columns

['num1', 'float1', 'str1', 'date1']

# Spark DataFrame

In [10]:

data = [('James','','Smith','1991-04-01','M',3000),
  ('Michael','Rose','','2000-05-19','M',4000),
  ('Robert','','Williams','1978-09-05','M',4000),
  ('Maria','Anne','Jones','1967-12-01','F',4000),
  ('Jen','Mary','Brown','1980-02-17','F',-1)
]

columns = ["firstname","middlename","lastname","dob","gender","salary"]

df = spark.createDataFrame(data=data, schema = columns)


In [11]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



In [12]:
df.take(3)

[Row(firstname='James', middlename='', lastname='Smith', dob='1991-04-01', gender='M', salary=3000),
 Row(firstname='Michael', middlename='Rose', lastname='', dob='2000-05-19', gender='M', salary=4000),
 Row(firstname='Robert', middlename='', lastname='Williams', dob='1978-09-05', gender='M', salary=4000)]

In [13]:
df.count()

5

# DataFrame from external data sources

In [15]:
df = spark.read.csv("S:/Datasets/csv/titanic.csv",header=True,inferSchema=True)
df.printSchema()


root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [17]:
df.show(4)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 4 rows

